In [1]:
import pandas as pd
from flask import Flask, jsonify, request
from threading import Thread
from collections import OrderedDict
import json

In [2]:
app = Flask(__name__)

In [3]:
@app.route('/predictions', methods=['GET'])
def get_predictions():
    try:
        df = pd.read_csv('prediksi_harian_2024_2025.csv')
    except FileNotFoundError:
        return jsonify({"error": "File prediksi tidak ditemukan"}), 404

    province_name = request.args.get('name')
    tanggal = request.args.get('tanggal')
    bulan = request.args.get('bulan')
    tahun = request.args.get('tahun')

    if province_name:
        df = df[df['name'] == province_name]
        if df.empty:
            return jsonify({"error": f"Data prediksi untuk provinsi {province_name} tidak ditemukan"}), 404

    if tanggal:
        df = df[df['tanggal'] == int(tanggal)] 
    if bulan:
        df = df[df['bulan'] == int(bulan)]  
    if tahun:
        df = df[df['tahun'] == int(tahun)]  

    if df.empty:
        return jsonify({"error": "Data prediksi tidak ditemukan untuk kriteria yang diberikan"}), 404

    predictions = df.to_dict(orient='records')

    formatted_predictions = []
    for pred in predictions:
        ordered_pred = OrderedDict({
            "name": pred["name"],
            "tanggal": pred["tanggal"],
            "bulan": pred["bulan"],
            "tahun": pred["tahun"],
            "cloudcover": pred["cloudcover"],
            "humidity": pred["humidity"],
            "precip": pred["precip"],
            "sealevelpressure": pred["sealevelpressure"],
            "temp": pred["temp"],
            "windspeed": pred["windspeed"]
        })
        formatted_predictions.append(ordered_pred)

    return json.dumps(formatted_predictions, indent=4)


In [4]:
def run_app():
    app.run(debug=False, use_reloader=False)

thread = Thread(target=run_app)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Oct/2024 20:32:13] "GET /predictions HTTP/1.1" 200 -
